In [2]:
import pandas as pd
import numpy as npcategory_encoders
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import requests
from mosaicml import * 
import category_encoders
from mosaicml.constants import MLModelFlavours

/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
!pip install category_encoders
# from mosaicml.constants import MLModelFlavours

In [3]:
# cars = pd.read_csv("/data/car_new.csv")
cars = pd.read_csv("/notebooks/notebooks/car_new.csv")
cars

,buying_num,maintenance_num,doors_num,persons_num,lugboot_num,safety_num,class_num
0,3,3,0,0,2,2,2
1,3,3,0,0,2,0,2
2,3,3,0,0,1,1,2
3,3,3,0,0,1,2,2
4,3,3,0,0,1,0,2
...,...,...,...,...,...,...,...
1722,1,1,3,2,1,2,1
1723,1,1,3,2,1,0,3
1724,1,1,3,2,0,1,2
1725,1,1,3,2,0,2,1


In [4]:
Y = cars['class_num']
X = cars.drop (['class_num'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.18, shuffle=True, random_state=25)
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1416, 6) (311, 6) (1416,) (311,)


In [5]:
first_tree = DecisionTreeClassifier()
model=first_tree.fit(X_train, y_train)
model

DecisionTreeClassifier()

In [6]:
y_pred = model.predict(X_test)
first_tree.score(X_test, y_test)
y_prob = model.predict_proba(X_test)[:,1]

In [7]:
@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data = pd.DataFrame(eval(payload))
    prediction = pd.Series(model.predict(data))
    return prediction.tolist()

In [8]:
req = requests.Request()
req.json = {"payload":X_train.head(1).to_json()}

sch = generate_schema(score,(model, req), X_train)
print(sch)

{'input': {'type': 'pd.core.frame.DataFrame', 'items': {'type': 'object', 'properties': {'buying_num': {'type': 'integer', 'format': 'int64'}, 'maintenance_num': {'type': 'integer', 'format': 'int64'}, 'doors_num': {'type': 'integer', 'format': 'int64'}, 'persons_num': {'type': 'integer', 'format': 'int64'}, 'lugboot_num': {'type': 'integer', 'format': 'int64'}, 'safety_num': {'type': 'integer', 'format': 'int64'}}}, 'example': [{'buying_num': 1, 'maintenance_num': 2, 'doors_num': 0, 'persons_num': 2, 'lugboot_num': 0, 'safety_num': 1}, {'buying_num': 3, 'maintenance_num': 2, 'doors_num': 2, 'persons_num': 2, 'lugboot_num': 2, 'safety_num': 2}]}, 'output': {'type': 'array', 'items': {'type': 'object'}, 'example': [2]}}


In [9]:
register_model(model, 
               score, 
               name='car_evaluation', 
               description='car evaluation classification model using sklearn', 
               flavour=MLModelFlavours.sklearn,
               schema=sch,
               y_true=y_test, 
               y_pred=y_pred,
               y_train=y_train, 
               y_test=y_test,
               x_train=X_train, 
               x_test=X_test, 
               prob=y_prob,
               labels=[0,1], 
               init_script="",
               model_type="classification",
               input_type="json", 
               explain_ai=True, 
               feature_names=list(X_train), 
               feature_ids=list(X_train),
               target_names=[0,1,2,3], 
               kyd=True, 
               kyd_score = False
              )

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


In [10]:
pip freeze

aiohttp==3.9.3
aiosignal==1.3.1
alembic==1.13.1
anyio==4.2.0
archspec==0.2.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
asttokens==2.4.1
async-generator==1.10
async-timeout==4.0.3
attrs==23.2.0
Babel==2.14.0
beautifulsoup4==4.12.3
bleach==4.1.0
blinker==1.7.0
boltons==23.0.0
bqplot==0.12.42
Brotli==1.0.9
category-encoders==2.6.3
certifi==2024.2.2
certipy==0.1.3
cffi==1.16.0
charset-normalizer==3.3.2
click==8.1.7
clickclick==20.10.2
cloudpickle==2.2.1
comm==0.2.1
conda==23.11.0
conda-content-trust==0.1.3
conda-libmamba-solver==23.12.0
conda-package-handling==2.2.0
conda-package-streaming==0.9.0
configparser==6.0.0
connexion==2.6.0
contourpy==1.2.0
cryptography==41.0.7
cycler==0.12.1
Cython==0.29.15
databricks-cli==0.18.0
deap==1.4.1
decorator==5.1.1
defusedxml==0.7.1
dill==0.3.8
distro==1.8.0
docker==6.1.3
ds-lime==0.1.1.27
entrypoints==0.4
exceptiongroup==1.2.0
executing==2.0.1
fastjsonschema==2.19.1
flasgger==0.9.5
flask==2.3.3
fonttools==4.49.0
frozenlist==1.4.1
gitdb==4.0.11
G